In [1]:
import os
import pathlib

import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

/home/pi/.local/lib/python3.7/site-packages/h5py/__init__.py:40: UserWarning: h5py is running against HDF5 1.10.6 when it was built against 1.10.4, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
import tensorflow_io as tfio

# 将一段频谱图转成梅尔频谱
def get_melcepstrum(spectrogram):
    # (124, 129, 1)
    melspectrogram = tfio.audio.melscale(
    spectrogram, rate=16000, mels=40, fmin=0, fmax=8000)
    melspectrogram = melspectrogram[..., tf.newaxis]# 新增一个aixs
    return melspectrogram

def get_spectrogram(waveform):
  # Zero-padding for an audio waveform with less than 16,000 samples.
  input_len = 16000
  waveform = waveform[:input_len]
  zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
  # Cast the waveform tensors' dtype to float32.
  waveform = tf.cast(waveform, dtype=tf.float32)
  # Concatenate the waveform with `zero_padding`, which ensures all audio
  # clips are of the same length.
  equal_length = tf.concat([waveform, zero_padding], 0)
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      equal_length, frame_length=1024, frame_step=512)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  # Batch Size，样本数（时间序列长度/frame_step=125），频率点数(样本窗口/2)，通道
  spectrogram = spectrogram[..., tf.newaxis]
  #melspectrogram = melspectrogram[..., tf.newaxis]
  return spectrogram

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['/home/pi/.local/lib/python3.7/site-packages/tensorflow_io-0.17.0-py3.7-linux-armv7l.egg/tensorflow_io/core/python/ops/libtensorflow_io.so']
caused by: ['/home/pi/.local/lib/python3.7/site-packages/tensorflow_io-0.17.0-py3.7-linux-armv7l.egg/tensorflow_io/core/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']

In [ ]:
import apa102

PIXELS_N = 3

rgb_led = apa102.APA102(num_led=PIXELS_N)

#RGB

rgb_led.set_pixel(0, 0, 0, 0) #没有信号

rgb_led.set_pixel(0, 0, 0, 0) #有打印正常声音信号

rgb_led.show()

# 以上为每次启动需要执行的初始化

In [ ]:
import cv2
from IPython.display import display, Image
import ipywidgets as widgets
import threading
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    cap.set(3,640) # adjust width
    cap.set(4,480) # adjust height
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

In [ ]:
import pyaudio
import wave
import numpy as np
import datetime
RESPEAKER_RATE = 16000
RESPEAKER_CHANNELS = 2
RESPEAKER_WIDTH = 2
# run getDeviceInfo.py to get index
RESPEAKER_INDEX = 0  # refer to input device id
CHUNK = 1024
RECORD_SECONDS = 1 #录音一秒
WAVE_OUTPUT_FILENAME = "./dataset/output_one_channel"

model = tf.keras.models.load_model('./SLA3dPrintAssitant For Raspi.h5')#加载模型

for i in range(0,3*60*60):#录音3h，生成3*3600个wav文件
    p = pyaudio.PyAudio()
    stream = p.open(
                rate=RESPEAKER_RATE,
                format=p.get_format_from_width(RESPEAKER_WIDTH),
                channels=RESPEAKER_CHANNELS,
                input=True,
                input_device_index=RESPEAKER_INDEX,)
    a=np.array([])
    frames = [] 
        
    while(1):
        try:    # Lookout for a keyboardInterrupt to stop the script
            #读取1秒 16000个点
            for i in range(0, int(RESPEAKER_RATE / CHUNK * RECORD_SECONDS)):
                data = stream.read(CHUNK,exception_on_overflow = False)
                # extract channel 0 data from 2 channels, if you want to extract channel 1, please change to [1::2]
                #a = np.fromstring(data,dtype=np.int16)[0::2]
                a = np.append(a,np.fromstring(data,dtype=np.int16)[0::2])
            #print("Wave shape:",a.shape)
            spectrogram = get_spectrogram(a)#信号转频谱
            del a
            a=np.array([])
            #print("spectrogram shape:",spectrogram.shape)
            melcepstrum = get_melcepstrum(spectrogram)#频谱转梅尔频谱
            #print("Model Input melcepstrum shape:",melcepstrum.shape)
            #print("Model Input melcepstrum Type:",type(melcepstrum))
            melcepstrum = tf.reshape(melcepstrum, (-1, 30, 40, 1))#调整维度，满足输入维度
            y_pred = np.argmax(model.predict(melcepstrum), axis=1) #0:Good 1:Background
            #print(y_pred)
            if(y_pred[0]==0):#有打印正常声音信号
                rgb_led.set_pixel(0, 0, 20, 0) #有打印正常声音信号
                rgb_led.show()
            else:
                rgb_led.set_pixel(0, 20, 0, 0) #没有信号
                rgb_led.show()
        except KeyboardInterrupt:
            vc.release()
            stream.stop_stream()
            stream.close()
            p.terminate()
            print("SLA 3d Printer Assitant Stop!")

# 以下为主进程

In [ ]:
import pyaudio
import wave
import numpy as np
import datetime
RESPEAKER_RATE = 16000
RESPEAKER_CHANNELS = 2
RESPEAKER_WIDTH = 2
# run getDeviceInfo.py to get index
RESPEAKER_INDEX = 0  # refer to input device id
CHUNK = 1024
RECORD_SECONDS = 1 #录音一秒
WAVE_OUTPUT_FILENAME = "./dataset/output_one_channel"

model = tf.keras.models.load_model('./KWS on Raspi.h5')#加载模型KWS on Raspi.h5 |SLA3dPrintAssitant For Raspi.h5

def kws_nerual_network():#神经网络识别声音
    p = pyaudio.PyAudio()
    stream = p.open(
                rate=RESPEAKER_RATE,
                format=p.get_format_from_width(RESPEAKER_WIDTH),
                channels=RESPEAKER_CHANNELS,
                input=True,
                input_device_index=RESPEAKER_INDEX,)
    a=np.array([])
    frames = [] 
        
    while(1):
        try:    # Lookout for a keyboardInterrupt to stop the script
            #读取1秒 16000个点
            for i in range(0, int(RESPEAKER_RATE / CHUNK * RECORD_SECONDS)):
                data = stream.read(CHUNK,exception_on_overflow = False)
                # extract channel 0 data from 2 channels, if you want to extract channel 1, please change to [1::2]
                #a = np.fromstring(data,dtype=np.int16)[0::2]
                a = np.append(a,np.fromstring(data,dtype=np.int16)[0::2])
            #print("Wave shape:",a.shape)
            spectrogram = get_spectrogram(a)#信号转频谱
            del a
            a=np.array([])
            #print("spectrogram shape:",spectrogram.shape)
            melcepstrum = get_melcepstrum(spectrogram)#频谱转梅尔频谱
            #print("Model Input melcepstrum shape:",melcepstrum.shape)
            #print("Model Input melcepstrum Type:",type(melcepstrum))
            melcepstrum = tf.reshape(melcepstrum, (-1, 30, 40, 1))#调整维度，满足输入维度
            y_pred = np.argmax(model.predict(melcepstrum), axis=1) #0:Good 1:Background
            print(y_pred)
            # R G B
            if(y_pred[0]==0):#down
                rgb_led.set_pixel(0, 5, 0, 0)
                #widgets.Label(value="Good")
                rgb_led.show()
            elif(y_pred[0]==1):#go
                rgb_led.set_pixel(0, 0, 5, 0) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==2):#left
                rgb_led.set_pixel(0, 0, 0, 5) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==3):#no
                rgb_led.set_pixel(1, 5, 0, 0) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==4):#right
                rgb_led.set_pixel(1, 0, 5, 0) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==5):#stop
                rgb_led.set_pixel(1, 0, 0, 5) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==6):#up
                rgb_led.set_pixel(2, 5, 0, 0) 
                #widgets.Label(value="Background")
                rgb_led.show()
            elif(y_pred[0]==7):#yes
                rgb_led.set_pixel(2, 0, 5, 0) 
                #widgets.Label(value="Background")
                rgb_led.show()
        except KeyboardInterrupt:
            vc.release()
            stream.stop_stream()
            stream.close()
            p.terminate()
            print("SLA 3d Printer Assitant Stop!")

import cv2
from IPython.display import display, Image
import ipywidgets as widgets
import threading
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    cap.set(3,160) # adjust width
    cap.set(4,80) # adjust height
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
thread1 = threading.Thread(target=view, args=(stopButton,))
thread2 = threading.Thread(target=kws_nerual_network,)

thread1.start()
thread2.start()


In [ ]:
import ipywidgets as widgets
widgets.Label(value="Background1")